<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/cadec_ask_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-4.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-4.validation.csv",header=None)
valid.head()

,0,1
0,6,diastolic dysfunction
1,8,ARTHRITIS
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-4.test.csv",header=None)
test.head()

,0,1
0,4,purple sploches around waist
1,6,stage 1 diastolic dysfunction
2,8,arthritis
3,12,shingles
4,16,flu


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'cadec_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'lipitor',
 'in',
 'of',
 'muscle',
 '.',
 'and',
 'to',
 ',',
 'my',
 'i',
 'the',
 'severe',
 'myalgia',
 'fatigue',
 'lower',
 'loss',
 '-',
 'weakness',
 'cramp',
 'limb',
 'stomach',
 'back',
 'a',
 'depression',
 'cramps',
 'memory',
 'gas',
 'leg',
 'joint',
 'upper',
 'legs',
 'pains',
 'it',
 'shoulder',
 'headache',
 'unable',
 'knee',
 'not',
 'neck',
 '/',
 'on',
 'was',
 'arthralgia',
 'foot',
 'aches',
 'excessive',
 'for',
 'numbness',
 'hip',
 'gastrointestinal',
 'all',
 'have',
 'arthritis',
 'like',
 'insomnia',
 'problems',
 'walk',
 'feeling',
 'bleeding',
 'with',
 'cholesterol',
 'arthrotec',
 'is',
 'nausea',
 'heart',
 'feet',
 'tired',
 'asthenia',
 'abdominal',
 'dizziness',
 'disease',
 'walking',
 'muscles',
 'very',
 'extreme',
 'this',
 'difficulty',
 'chest',
 'that',
 'swelling',
 'hair',
 'hand',
 'vision',
 'ache',
 'stiffness',
 'burning',
 'increased'

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.214265,2.684796,0.546136,18:28


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.790300,2.634629,0.551807,18:36


In [25]:
learn.save_encoder('cadec_ask_fold4_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('cadec_ask_fold4_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.960866,3.535005,0.364734,01:11


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.839100,2.822502,0.502415,01:20


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.390575,2.378590,0.583333,03:34


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.032043,2.088089,0.634058,03:12
1,1.704927,1.967600,0.642512,03:18


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.552857,1.781871,0.678744,03:23
1,1.326523,1.676098,0.699275,03:17


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.269224,1.549679,0.721014,03:30
1,1.068344,1.499024,0.733092,03:37


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.055958,1.422552,0.739130,03:35
1,0.877657,1.373379,0.742754,03:26


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.820105,1.330679,0.754831,03:26
1,0.761127,1.294683,0.769324,03:27


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.671212,1.272330,0.766908,03:22
1,0.683054,1.255298,0.771739,03:09
2,0.574152,1.185923,0.791063,03:16
3,0.511572,1.173516,0.789855,03:25


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.475634,1.228707,0.782609,03:25
1,0.509505,1.169981,0.783816,03:17
2,0.426664,1.232780,0.794686,03:12
3,0.372424,1.138012,0.799517,03:26


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.369944,1.183464,0.804348,03:17
1,0.413191,1.188487,0.794686,03:22
2,0.370129,1.181999,0.811594,03:24
3,0.301325,1.105579,0.816425,03:06


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.310530,1.207618,0.807971,03:18
1,0.343754,1.111421,0.814010,03:08
2,0.301910,1.163076,0.815217,03:17
3,0.258208,1.191943,0.820048,03:14


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.249036,1.210341,0.815217,03:06
1,0.328889,1.167747,0.821256,03:22
2,0.241159,1.137225,0.834541,03:21
3,0.215596,1.166570,0.832126,03:24


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.242175,1.224948,0.822464,03:18
1,0.265685,1.259827,0.818841,03:09
2,0.245929,1.166201,0.828502,03:07
3,0.205621,1.194117,0.832126,03:24


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.211625,1.218405,0.818841,03:04
1,0.274133,1.274535,0.822464,03:26
2,0.218022,1.226477,0.832126,03:17
3,0.178583,1.213690,0.830918,03:16


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.212975,1.331282,0.823672,03:19
1,0.181058,1.209467,0.829710,03:16


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.201434,1.312363,0.818841,03:24
1,0.176533,1.290628,0.827295,03:12


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.191565,1.321366,0.821256,03:26


In [44]:
# save the best model

learn.save_encoder('cadec_ask_mixed_fold4')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[670, 6, 8, 12, 16, 16, 27, 27, 27, 29, 414, 414, 414, 414, 414, 32, 37, 37, 37, 37, 37, 37, 37, 42, 48, 49, 49, 49, 49, 49, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 474, 541, 474, 541, 226, 395, 61, 61, 62, 64, 76, 76, 76, 614, 289, 79, 94, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 80, 80, 80, 268, 91, 91, 92, 92, 92, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 905, 94, 94, 96, 96, 96, 100, 100, 100, 107, 108, 108, 243, 111, 111, 289, 111, 111, 111, 211, 113, 116, 179, 120, 122, 122, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 129, 131, 139, 140, 143, 886, 157, 736, 165, 178, 166, 166, 291, 172, 172, 172, 172, 172, 172, 172, 427, 165, 182, 182, 182, 182, 182, 182, 182, 623, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 186, 187, 187, 188, 189, 189, 293, 190, 190, 190, 190, 190, 289, 289, 197, 197, 205, 206, 207, 208, 210, 214, 299, 214, 214, 214, 214, 471, 

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

637
0.7355658198614319
